In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold

In [10]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [11]:
podcasts_segments = np.load('podcast_segmented.npy')
ads_segments = np.load('adBinary_segmented.npy')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(np.array(podcasts_segments),
                                                    np.array(ads_segments),
                                                    test_size=0.10,
                                                    random_state=42)


X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                y_test, 
                                                test_size=0.5,
                                                random_state=42)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2],1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)
print(X_train.shape)

(196051, 128, 10)
(10892, 128, 10)
(10892, 128, 10)
(196051, 128, 10, 1)


In [13]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 10, 1)))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.summary()

def get_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 10, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))


    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.0001),
              metrics='accuracy')

    return model

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 126, 8, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 4, 32)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8064)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                516160    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 516,545
Trainable params: 516,545
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.0001),
              metrics='accuracy')
              
#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))


In [15]:
def get_evaluation(model, X_train, X_test, y_train, y_test):
    history = model.fit(X_train,y_train,epochs=50)
    return model.evaluate(X_test, y_test)

In [16]:
history_arr = []

for train_index, test_index in kfold.split(np.array(podcasts_segments), np.array(ads_segments)):
    X_train, X_test, y_train, y_test = np.array(podcasts_segments)[train_index], np.array(podcasts_segments)[test_index], \
                                        np.array(ads_segments)[train_index], np.array(ads_segments)[test_index]
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)
    
    history_arr.append(get_evaluation(get_model(),X_train,X_test,y_train,y_test))

Epoch 1/2
5106/5106 [==============================] - 70s 14ms/step - loss: 2.2147 - accuracy: 0.7378
Epoch 2/2
1702/1702 [==============================] - 5s 3ms/step - loss: 1.4397 - accuracy: 0.7074
Epoch 1/2
5106/5106 [==============================] - 67s 13ms/step - loss: 1.6647 - accuracy: 0.78000s - loss: 1.6649 - accuracy: 0.78
Epoch 2/2
1702/1702 [==============================] - 6s 3ms/step - loss: 3.6878 - accuracy: 0.6661
Epoch 1/2
5106/5106 [==============================] - 68s 13ms/step - loss: 2.4717 - accuracy: 0.7423
Epoch 2/2
1702/1702 [==============================] - 5s 3ms/step - loss: 1.7904 - accuracy: 0.7441
Epoch 1/2
5106/5106 [==============================] - 66s 13ms/step - loss: 2.5428 - accuracy: 0.7255
Epoch 2/2
1702/1702 [==============================] - 4s 3ms/step - loss: 1.2813 - accuracy: 0.8139


In [20]:
history_arr

[[1.4396694898605347, 0.7073761820793152],
 [3.687818765640259, 0.666079044342041],
 [1.7904168367385864, 0.74411940574646],
 [1.281252145767212, 0.813948392868042]]

In [18]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

NameError: name 'history' is not defined